In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
)

from sklearn.feature_selection import SelectKBest

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import os

print(os.getcwd())

print(os.listdir(os.getcwd()))

C:\Users\jlim7\OneDrive\Documents\GitHub\Capstone2
['.git', '.gitignore', '.ipynb_checkpoints', '42343_72434_bundle_archive', '42343_72434_bundle_archive.zip', 'Classification Models Second Round.ipynb', 'Initial Classification Models.ipynb', 'Initial EDA.ipynb', 'LICENSE', 'README.md', 'WA_Fn-UseC_-Telco-Customer-Churn.xlsx']


<IPython.core.display.Javascript object>

In [3]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(np.array(x), i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

<IPython.core.display.Javascript object>

In [4]:
pd.set_option("display.max_columns", None)
churn = pd.read_excel("WA_Fn-UseC_-Telco-Customer-Churn.xlsx")
churn.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<IPython.core.display.Javascript object>

In [5]:
churn = pd.get_dummies(churn, columns=["Churn"], drop_first=True)
churn = churn.drop(columns=["customerID", "TotalCharges"])

<IPython.core.display.Javascript object>

In [6]:
X = churn.drop(columns=["Churn_Yes"])
y = churn["Churn_Yes"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13, stratify=y
)

<IPython.core.display.Javascript object>

In [7]:
X.isna().mean()

gender              0.0
SeniorCitizen       0.0
Partner             0.0
Dependents          0.0
tenure              0.0
PhoneService        0.0
MultipleLines       0.0
InternetService     0.0
OnlineSecurity      0.0
OnlineBackup        0.0
DeviceProtection    0.0
TechSupport         0.0
StreamingTV         0.0
StreamingMovies     0.0
Contract            0.0
PaperlessBilling    0.0
PaymentMethod       0.0
MonthlyCharges      0.0
dtype: float64

<IPython.core.display.Javascript object>

In [8]:
churn["OnlineSecurity"] = churn["OnlineSecurity"] == "Yes"
churn["OnlineBackup"] = churn["OnlineBackup"] == "Yes"
churn["DeviceProtection"] = churn["DeviceProtection"] == "Yes"
churn["TechSupport"] = churn["TechSupport"] == "Yes"
churn["StreamingTV"] = churn["StreamingTV"] == "Yes"
churn["StreamingMovies"] = churn["StreamingMovies"] == "Yes"

<IPython.core.display.Javascript object>

In [9]:
col = [
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]

for c in col:
    churn[col] = churn[col].astype(int)

<IPython.core.display.Javascript object>

In [10]:
bin_cols = [
    "SeniorCitizen",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]
cat_cols = [
    "gender",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
]
drop_cats = [
    "Male",
    "No",
    "No",
    "Yes",
    "No",
    "Fiber optic",
    "Month-to-month",
    "Yes",
    "Electronic check",
]
num_cols = ["tenure", "MonthlyCharges"]

<IPython.core.display.Javascript object>

In [11]:
preprocessing = ColumnTransformer(
    [
        ("scale", StandardScaler(), num_cols),
        ("one_hot", OneHotEncoder(drop=drop_cats), cat_cols),
    ],
    remainder="passthrough",
)

<IPython.core.display.Javascript object>

In [12]:
encoder = OneHotEncoder(drop=drop_cats, sparse=False)
X_preprocessed = pd.DataFrame(encoder.fit_transform(X_train[cat_cols]))
X_preprocessed.columns = encoder.get_feature_names(cat_cols)
print_vif(X_preprocessed)

VIF results
-------------------------------
const                                      7.179093
gender_Female                              1.001534
Partner_Yes                                1.386883
Dependents_Yes                             1.338551
PhoneService_No                                 inf
MultipleLines_No phone service                  inf
MultipleLines_Yes                          1.325600
InternetService_DSL                        1.683530
InternetService_No                         1.859399
Contract_One year                          1.213636
Contract_Two year                          1.486182
PaperlessBilling_No                        1.203400
PaymentMethod_Bank transfer (automatic)    1.454779
PaymentMethod_Credit card (automatic)      1.469735
PaymentMethod_Mailed check                 1.639003
dtype: float64
-------------------------------



C:\Users\jlim7\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


<IPython.core.display.Javascript object>

In [13]:
X_preprocessed.shape

(5634, 14)

<IPython.core.display.Javascript object>

In [14]:
print_vif(X.select_dtypes("number"))

VIF results
-------------------------------
const             6.288773
SeniorCitizen     1.052649
tenure            1.067205
MonthlyCharges    1.121267
dtype: float64
-------------------------------



<IPython.core.display.Javascript object>

According to the vif results there aren't any columns that have high colinearity. In order to reduce features I've decided to first use SelectKBest and choose 15 of the columns. 

In [15]:
# X = X.reset_index(drop=True)
# churn = churn.reset_index(drop=True)
# X_preprocessed = X_preprocessed.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [16]:
# New Data frame with features as numerics
# result = pd.concat(
#     [X_preprocessed, X[num_cols], churn[bin_cols], churn["Churn_Yes"]], axis=1,
# )

<IPython.core.display.Javascript object>

In [17]:
# result.isna().mean()

<IPython.core.display.Javascript object>

In [18]:
churn = pd.get_dummies(churn, columns=cat_cols, drop_first=True)

<IPython.core.display.Javascript object>

In [19]:
churn.head()

,SeniorCitizen,tenure,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MonthlyCharges,Churn_Yes,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0,1,0,0,0,0,29.85,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0
1,0,34,1,0,1,0,0,0,56.95,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1
2,0,2,1,1,0,0,0,0,53.85,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1
3,0,45,1,0,1,1,0,0,42.30,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0
4,0,2,0,0,0,0,0,0,70.70,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0


<IPython.core.display.Javascript object>

In [28]:
X = churn.drop(columns=["Churn_Yes"])
y = churn["Churn_Yes"]
feature_names = list(X.columns.values)

<IPython.core.display.Javascript object>

In [23]:
selector = SelectKBest(k=15).fit(X, y)
X = selector.transform(X)
scores = selector.scores_

<IPython.core.display.Javascript object>

In [24]:
X.shape

(7043, 15)

<IPython.core.display.Javascript object>

In [29]:
mask = selector.get_support()  # list of booleans
new_features = []  # The list of your K best features

for bool, feature in zip(mask, feature_names):
    if bool:
        new_features.append(feature)

<IPython.core.display.Javascript object>

In [30]:
X.head()

,SeniorCitizen,tenure,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MonthlyCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0,1,0,0,0,0,29.85,0,1,0,0,1,0,0,0,0,0,1,0,1,0
1,0,34,1,0,1,0,0,0,56.95,1,0,0,1,0,0,0,0,1,0,0,0,0,1
2,0,2,1,1,0,0,0,0,53.85,1,0,0,1,0,0,0,0,0,0,1,0,0,1
3,0,45,1,0,1,1,0,0,42.30,1,0,0,0,1,0,0,0,1,0,0,0,0,0
4,0,2,0,0,0,0,0,0,70.70,0,0,0,1,0,0,1,0,0,0,1,0,1,0


<IPython.core.display.Javascript object>

In [31]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13, stratify=y
)

<IPython.core.display.Javascript object>

In [33]:
preprocessing = ColumnTransformer(
    [("scale", StandardScaler(), num_cols),], remainder="passthrough",
)

<IPython.core.display.Javascript object>

# K Nearest Neighbor Classifier

In [35]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("knn", KNeighborsClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['tenure',
                                                   'MonthlyCharges'])],
                                   verbose=False)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                            

<IPython.core.display.Javascript object>

In [36]:
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

Train score: 0.8375931842385517
Test score: 0.7863733144073811


<IPython.core.display.Javascript object>

In [37]:
y_pred = pipeline.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[901 134]
 [167 207]]
              precision    recall  f1-score   support

           0       0.84      0.87      0.86      1035
           1       0.61      0.55      0.58       374

    accuracy                           0.79      1409
   macro avg       0.73      0.71      0.72      1409
weighted avg       0.78      0.79      0.78      1409



<IPython.core.display.Javascript object>

#### Add a grid search to optimize KNN model

In [38]:
grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11, 13, 15, 17, 19, 21],
    "knn__weights": ["distance", "uniform"],
}

model = GridSearchCV(pipeline, grid, cv=2, n_jobs=-1)
model.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.0s


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['tenure',
                                                                       

<IPython.core.display.Javascript object>

In [39]:
model.best_params_

{'knn__n_neighbors': 19, 'knn__weights': 'uniform'}

<IPython.core.display.Javascript object>

In [40]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

Train score: 0.8130990415335463
Test score: 0.7963094393186657


<IPython.core.display.Javascript object>

In [41]:
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[911 124]
 [163 211]]
              precision    recall  f1-score   support

           0       0.85      0.88      0.86      1035
           1       0.63      0.56      0.60       374

    accuracy                           0.80      1409
   macro avg       0.74      0.72      0.73      1409
weighted avg       0.79      0.80      0.79      1409



<IPython.core.display.Javascript object>

# Support Vector Classifier

In [43]:
pipeline = Pipeline([("preprocessing", preprocessing), ("SVC", SVC())], verbose=True)

pipeline.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing SVC, total=   0.7s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['tenure',
                                                   'MonthlyCharges'])],
                                   verbose=False)),
                ('SVC',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probabili

<IPython.core.display.Javascript object>

In [44]:
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

Train score: 0.8161164359247426
Test score: 0.8168914123491838


<IPython.core.display.Javascript object>

In [45]:
y_pred = pipeline.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[964  71]
 [187 187]]
              precision    recall  f1-score   support

           0       0.84      0.93      0.88      1035
           1       0.72      0.50      0.59       374

    accuracy                           0.82      1409
   macro avg       0.78      0.72      0.74      1409
weighted avg       0.81      0.82      0.80      1409



<IPython.core.display.Javascript object>

#### Add a grid search to optimize SVC model

In [46]:
grid = {
    "SVC__kernel": ["linear", "rbf", "poly"],
    "SVC__C": [1, 10, 100, 1000],
    "SVC__degree": [3, 5, 7, 10],
}

model = GridSearchCV(pipeline, grid, cv=2, n_jobs=-1)
model.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing SVC, total=   0.7s


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['tenure',
                                                                       

<IPython.core.display.Javascript object>

# Random Forest Classifier

In [49]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("rf", RandomForestClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing rf, total=   0.4s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['tenure',
                                                   'MonthlyCharges'])],
                                   verbose=False)),
                ('rf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                       

<IPython.core.display.Javascript object>

In [50]:
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

Train score: 0.9973375931842385
Test score: 0.8055358410220014


<IPython.core.display.Javascript object>

In [51]:
y_pred = pipeline.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[941  94]
 [180 194]]
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      1035
           1       0.67      0.52      0.59       374

    accuracy                           0.81      1409
   macro avg       0.76      0.71      0.73      1409
weighted avg       0.80      0.81      0.80      1409



<IPython.core.display.Javascript object>

#### Add a grid search to optimize Random Forest model

In [52]:
grid = {
    "rf__max_depth": [3, 5, 7, 10, 15],
    "rf__n_estimators": [1, 10, 100, 1000, 10000],
    "rf__min_samples_leaf": [1, 3, 5, 7, 10],
    "rf__criterion": ["gini", "entropy"],
}

model = GridSearchCV(pipeline, grid, cv=2, n_jobs=-1)
model.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing rf, total=   0.3s


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['tenure',
                                                                       

<IPython.core.display.Javascript object>

In [53]:
model.best_params_

{'rf__criterion': 'entropy',
 'rf__max_depth': 7,
 'rf__min_samples_leaf': 10,
 'rf__n_estimators': 100}

<IPython.core.display.Javascript object>

In [54]:
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

Train score: 0.8107916222932198
Test score: 0.7955997161107168


<IPython.core.display.Javascript object>

In [55]:
y_pred = model.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[958  77]
 [211 163]]
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      1035
           1       0.68      0.44      0.53       374

    accuracy                           0.80      1409
   macro avg       0.75      0.68      0.70      1409
weighted avg       0.78      0.80      0.78      1409



<IPython.core.display.Javascript object>

# Gradient Boosted Classifier

In [56]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("xgb", XGBClassifier())], verbose=-1
)

pipeline.fit(X_train, y_train)

[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing xgb, total=   0.4s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['tenure',
                                                   'MonthlyCharges'])],
                                   verbose=False)),
                ('xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsampl...
                               interaction_constraints='',
                               learning_rate=0.300000012, m

<IPython.core.display.Javascript object>

In [57]:
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

Train score: 0.9298899538516152
Test score: 0.7955997161107168


<IPython.core.display.Javascript object>

In [58]:
y_pred = pipeline.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[923 112]
 [176 198]]
              precision    recall  f1-score   support

           0       0.84      0.89      0.87      1035
           1       0.64      0.53      0.58       374

    accuracy                           0.80      1409
   macro avg       0.74      0.71      0.72      1409
weighted avg       0.79      0.80      0.79      1409



<IPython.core.display.Javascript object>

#### Add a grid search to optimize XGB model

In [ ]:
grid = {
    "xgb__subsample": [0.5, 0.75, 1.0],
    "xgb__colsample_bytree": [0.5, 0.75, 1.0],
    "xgb__max_depth": [5, 7, 10],
    "xgb__n_trees": [100, 150, 200],
    "xgb__learning_rate": [0.2, 0.4, 0.6, 0.8, 1],
}
model = GridSearchCV(pipeline, grid, cv=2, n_jobs=-1)
model.fit(X_train, y_train)